In [1]:
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
import requests
import time
from selenium import webdriver

# ***********************************PLEASE READ***********************************
#### This will open an extra chrome tab just used to parse some of the data that I couldnt get normally from the website directly because it parsed back "None" Values for some elements even though it existed, therefor I couldn't get all the data I wanted to collect for this project. the chrome tab allows me read those without restrictions and it solves the "None" Value problem

## IMPORTANT: 
### 1) Please dont close the tab or the project will break, you can close the tab when you stop running the code.
### 2) Please when you run the app, make sure to STOP RUNNING the code within a few seconds, as the code will continue to run until its done with all the pages. It has to loop through 33,545 titles which can take a long time.

In [37]:
#test code
"""
*****************************************************PLEASE READ*****************************************************
This will open an extra chrome tab just used to parse some of the data that I couldnt get normally from the website 
directly because it parsed back "None" Values for some elements even though it existed, therefor I couldn't get all the data 
I wanted to collect for this project. the chrome tab allows me read those without restrictions and it solves 
the "None" Value problem

IMPORTANT: Please dont close the tab or the project will break, the tab will close automatically when it is done loading.
"""
driver = webdriver.Chrome()
url = 'https://www.imdb.com/search/title/?title_type=video_game&sort=user_rating,desc'
# driver.maximize_window()
driver.get(url)
content = driver.page_source.encode('utf-8').strip()
""""""
soup = BeautifulSoup(content,"html.parser")

def data_create():
    data = {
        'title': [],
        'release_date': [],
        'maturity_rating': [],
        'genre': [],
        'rating': [],
        'directors': [],
        'stars': [],
        'votes': []
    }
    global df
    df = pd.DataFrame(data= data)
data_create()


#func used to extract data by inputting the soup and the thing you want to find => "selector"
def extract_data(soup, selector, class_selector):
    try:
        return soup.find(selector, class_= class_selector).text.strip()
    except AttributeError:
        return None


while True:
    game_info = soup.find_all("div", class_="lister-item-content")
    
    for i in game_info:
        #gets game name
        try:
            game_name = (i.find("h3", class_="lister-item-header")).a.text.strip()
        except AttributeError:
            game_name = None
        
        #gets release date
        game_release_date = extract_data(i, "span", "lister-item-year text-muted unbold")
        game_release_date = game_release_date.replace("(", "")
        game_release_date = game_release_date.replace(")", "")
        game_release_date = int(game_release_date.replace("Video Game", "").strip())
        
        #gets maturity rating
        game_maturity = extract_data(i, "span", "certificate")

        #gets genre and split it into an array
        game_genre = extract_data(i, "span", "genre")
        game_genre = np.array(list(game_genre.split(",")))
        
        #get rating
        try:
            game_rating = float(i.find("div", class_="inline-block ratings-imdb-rating").strong.text.strip())
        except AttributeError:
            game_rating = None
        
        #get Director & Stars and will split them
        #get both
        game_director_and_stars = extract_data(i, "p", "")
        game_director = None
        game_stars = None
        #if both are found
        if game_director_and_stars.find("|")!=-1:
            game_director_full_text, separator, game_stars_full_text = game_director_and_stars.partition("|")

            game_director_full_text = game_director_full_text.strip()
            game_director_full_text = ''.join(game_director_full_text.splitlines())
            game_stars_full_text = game_stars_full_text.strip()
            game_stars_full_text = ''.join(game_stars_full_text.splitlines())

            director_text, separator, game_director = game_director_full_text.partition(":")
            star_text, separator, game_stars = game_stars_full_text.partition(":")

            #finds if there are many directors/actors or just one
            #there are multiple
            if game_director.find(",")!=1:
                game_director = np.array(list(game_director.split(",")))
            #there is only 1
            else:
                game_director = np.array(list(game_director))

            if game_stars.find(",")!=1:
                game_stars = np.array(list(game_stars.split(",")))
            else:
                game_stars = np.array(list(game_stars))

        #didnt find both, will try to find one        
        #if found Director
        elif game_director_and_stars.find("Director")!=-1:
            game_director_and_stars = ''.join(game_director_and_stars.splitlines())
            director_text, separator, game_director = game_director_and_stars.partition(":")
            game_director = game_director.replace("\n", "")
            #finds if there are many directors/actors or just one
            #there are multiple directors
            if game_director.find(",")!=1:
                game_director = np.array(list(game_director.split(",")))
            #there is only 1 director
            else:
                game_director = np.array(list(game_director))
            game_stars = None
        
        #if found Star
        elif game_director_and_stars.find("Star")!=-1:
            star_text, separator, game_stars = game_director_and_stars.partition(":")
            game_stars = game_stars.replace("\n", "")
            #finds if there are many directors/actors or just one
            #there are multiple
            if game_stars.find(",")!=1:
                game_stars = np.array(list(game_stars.split(",")))
            #there is only 1
            else:
                game_stars = np.array(list(game_stars))
            game_director = None
        else:
            game_director = None
            game_stars = None
        
        
        #get votes
        game_votes = extract_data(i, "p", "sort-num_votes-visible")
        game_votes = ''.join(game_votes.splitlines())
        deleted_text, separator, game_votes = game_votes.partition(":")
        game_votes = int(game_votes.replace(",", "").strip())
        
        
        #adds game details to dataframe
        df.loc[len(df.index)] = [game_name, game_release_date, game_maturity, game_genre, game_rating, game_director, game_stars, game_votes]
#         display(df)
    
    
    #find button to click on next page
    uncompleted_next_page_url = soup.find("a", class_="lister-page-next next-page").get("href")
    completed_next_page_url = "https://www.imdb.com/" + uncompleted_next_page_url
    
    print(completed_next_page_url)

    #replaces old url to completed_next_page_url
    url = completed_next_page_url
    driver.get(url)
    content = driver.page_source.encode('utf-8').strip()
    
#     html_request = requests.get(url).text
    soup = BeautifulSoup(content, "html.parser")


driver.quit()

D:\Anaconda\lib\site-packages\pandas\core\dtypes\cast.py:948: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  element = np.asarray(element)


,title,release_date,maturity_rating,genre,rating,directors,stars,votes
0,The Last of Us: Part I,2022,M,"[Action, Adventure, Drama]",9.8,"[Matthew Gallant, Bruce Straley]","[Ashley Johnson, Troy Baker, Hana Hayes, Je...",3495


,title,release_date,maturity_rating,genre,rating,directors,stars,votes
0,The Last of Us: Part I,2022,M,"[Action, Adventure, Drama]",9.8,"[Matthew Gallant, Bruce Straley]","[Ashley Johnson, Troy Baker, Hana Hayes, Je...",3495
1,The Witcher 3: Wild Hunt - Blood and Wine,2016,M,"[Action, Adventure, Drama]",9.8,[Konrad Tomaszkiewicz],"[Doug Cockle, MyAnna Buring, Antonia Bernath...",8789


,title,release_date,maturity_rating,genre,rating,directors,stars,votes
0,The Last of Us: Part I,2022,M,"[Action, Adventure, Drama]",9.8,"[Matthew Gallant, Bruce Straley]","[Ashley Johnson, Troy Baker, Hana Hayes, Je...",3495
1,The Witcher 3: Wild Hunt - Blood and Wine,2016,M,"[Action, Adventure, Drama]",9.8,[Konrad Tomaszkiewicz],"[Doug Cockle, MyAnna Buring, Antonia Bernath...",8789
2,The Last of Us,2013,M,"[Action, Adventure, Drama]",9.7,"[Neil Druckmann, Bruce Straley]","[Troy Baker, Ashley Johnson, Hana Hayes, Je...",68852


,title,release_date,maturity_rating,genre,rating,directors,stars,votes
0,The Last of Us: Part I,2022,M,"[Action, Adventure, Drama]",9.8,"[Matthew Gallant, Bruce Straley]","[Ashley Johnson, Troy Baker, Hana Hayes, Je...",3495
1,The Witcher 3: Wild Hunt - Blood and Wine,2016,M,"[Action, Adventure, Drama]",9.8,[Konrad Tomaszkiewicz],"[Doug Cockle, MyAnna Buring, Antonia Bernath...",8789
2,The Last of Us,2013,M,"[Action, Adventure, Drama]",9.7,"[Neil Druckmann, Bruce Straley]","[Troy Baker, Ashley Johnson, Hana Hayes, Je...",68852
3,God of War: Ragnarök,2022,M,"[Action, Adventure, Drama]",9.7,[Eric Williams],"[Christopher Judge, Sunny Suljic, Danielle B...",11538


...

,title,release_date,maturity_rating,genre,rating,directors,stars,votes
0,The Last of Us: Part I,2022,M,"[Action, Adventure, Drama]",9.8,"[Matthew Gallant, Bruce Straley]","[Ashley Johnson, Troy Baker, Hana Hayes, Je...",3495
1,The Witcher 3: Wild Hunt - Blood and Wine,2016,M,"[Action, Adventure, Drama]",9.8,[Konrad Tomaszkiewicz],"[Doug Cockle, MyAnna Buring, Antonia Bernath...",8789
2,The Last of Us,2013,M,"[Action, Adventure, Drama]",9.7,"[Neil Druckmann, Bruce Straley]","[Troy Baker, Ashley Johnson, Hana Hayes, Je...",68852
3,God of War: Ragnarök,2022,M,"[Action, Adventure, Drama]",9.7,[Eric Williams],"[Christopher Judge, Sunny Suljic, Danielle B...",11538
4,Red Dead Redemption II,2018,M,"[Action, Adventure, Crime]",9.7,None,"[Benjamin Byron Davis, Roger Clark, Rob Wiet...",42785
5,The Witcher 3: Wild Hunt,2015,M,"[Action, Adventure, Drama]",9.7,[Konrad Tomaszkiewicz],"[Doug Cockle, Denise Gough, Jo Wyatt, Jaimi...",29095
6,Mass Effect: Legendary Edition,2021,M,"[Action, Adventure, Drama]",9.6,"[Casey Hudson, Mac Walters]","[Steven Barr, Kimberly Brooks, Keith David, ...",2209
7,Metal Gear Solid,1998,M,"[Action, Adventure, Drama]",9.6,[Hideo Kojima],"[David Hayter, Patric Zimmerman, Peter Lurie...",12111
8,The Legend of Zelda: Ocarina of Time,1998,E,"[Action, Adventure, Fantasy]",9.6,"[Eiji Aonuma, Yoichi Yamada, Yoshiaki Koizum...","[Nobuyuki Hiyama, Fujiko Takimoto, Jun Mizus...",9977
9,Grand Theft Auto V,2013,M,"[Action, Comedy, Crime]",9.5,None,"[Shawn Fonteno, Ned Luke, Steven Ogg, Geral...",64452


,title,release_date,maturity_rating,genre,rating,directors,stars,votes
0,The Last of Us: Part I,2022,M,"[Action, Adventure, Drama]",9.8,"[Matthew Gallant, Bruce Straley]","[Ashley Johnson, Troy Baker, Hana Hayes, Je...",3495
1,The Witcher 3: Wild Hunt - Blood and Wine,2016,M,"[Action, Adventure, Drama]",9.8,[Konrad Tomaszkiewicz],"[Doug Cockle, MyAnna Buring, Antonia Bernath...",8789
2,The Last of Us,2013,M,"[Action, Adventure, Drama]",9.7,"[Neil Druckmann, Bruce Straley]","[Troy Baker, Ashley Johnson, Hana Hayes, Je...",68852
3,God of War: Ragnarök,2022,M,"[Action, Adventure, Drama]",9.7,[Eric Williams],"[Christopher Judge, Sunny Suljic, Danielle B...",11538
4,Red Dead Redemption II,2018,M,"[Action, Adventure, Crime]",9.7,None,"[Benjamin Byron Davis, Roger Clark, Rob Wiet...",42785
5,The Witcher 3: Wild Hunt,2015,M,"[Action, Adventure, Drama]",9.7,[Konrad Tomaszkiewicz],"[Doug Cockle, Denise Gough, Jo Wyatt, Jaimi...",29095
6,Mass Effect: Legendary Edition,2021,M,"[Action, Adventure, Drama]",9.6,"[Casey Hudson, Mac Walters]","[Steven Barr, Kimberly Brooks, Keith David, ...",2209
7,Metal Gear Solid,1998,M,"[Action, Adventure, Drama]",9.6,[Hideo Kojima],"[David Hayter, Patric Zimmerman, Peter Lurie...",12111
8,The Legend of Zelda: Ocarina of Time,1998,E,"[Action, Adventure, Fantasy]",9.6,"[Eiji Aonuma, Yoichi Yamada, Yoshiaki Koizum...","[Nobuyuki Hiyama, Fujiko Takimoto, Jun Mizus...",9977
9,Grand Theft Auto V,2013,M,"[Action, Comedy, Crime]",9.5,None,"[Shawn Fonteno, Ned Luke, Steven Ogg, Geral...",64452


,title,release_date,maturity_rating,genre,rating,directors,stars,votes
0,The Last of Us: Part I,2022,M,"[Action, Adventure, Drama]",9.8,"[Matthew Gallant, Bruce Straley]","[Ashley Johnson, Troy Baker, Hana Hayes, Je...",3495
1,The Witcher 3: Wild Hunt - Blood and Wine,2016,M,"[Action, Adventure, Drama]",9.8,[Konrad Tomaszkiewicz],"[Doug Cockle, MyAnna Buring, Antonia Bernath...",8789
2,The Last of Us,2013,M,"[Action, Adventure, Drama]",9.7,"[Neil Druckmann, Bruce Straley]","[Troy Baker, Ashley Johnson, Hana Hayes, Je...",68852
3,God of War: Ragnarök,2022,M,"[Action, Adventure, Drama]",9.7,[Eric Williams],"[Christopher Judge, Sunny Suljic, Danielle B...",11538
4,Red Dead Redemption II,2018,M,"[Action, Adventure, Crime]",9.7,None,"[Benjamin Byron Davis, Roger Clark, Rob Wiet...",42785
5,The Witcher 3: Wild Hunt,2015,M,"[Action, Adventure, Drama]",9.7,[Konrad Tomaszkiewicz],"[Doug Cockle, Denise Gough, Jo Wyatt, Jaimi...",29095
6,Mass Effect: Legendary Edition,2021,M,"[Action, Adventure, Drama]",9.6,"[Casey Hudson, Mac Walters]","[Steven Barr, Kimberly Brooks, Keith David, ...",2209
7,Metal Gear Solid,1998,M,"[Action, Adventure, Drama]",9.6,[Hideo Kojima],"[David Hayter, Patric Zimmerman, Peter Lurie...",12111
8,The Legend of Zelda: Ocarina of Time,1998,E,"[Action, Adventure, Fantasy]",9.6,"[Eiji Aonuma, Yoichi Yamada, Yoshiaki Koizum...","[Nobuyuki Hiyama, Fujiko Takimoto, Jun Mizus...",9977
9,Grand Theft Auto V,2013,M,"[Action, Comedy, Crime]",9.5,None,"[Shawn Fonteno, Ned Luke, Steven Ogg, Geral...",64452


,title,release_date,maturity_rating,genre,rating,directors,stars,votes
0,The Last of Us: Part I,2022,M,"[Action, Adventure, Drama]",9.8,"[Matthew Gallant, Bruce Straley]","[Ashley Johnson, Troy Baker, Hana Hayes, Je...",3495
1,The Witcher 3: Wild Hunt - Blood and Wine,2016,M,"[Action, Adventure, Drama]",9.8,[Konrad Tomaszkiewicz],"[Doug Cockle, MyAnna Buring, Antonia Bernath...",8789
2,The Last of Us,2013,M,"[Action, Adventure, Drama]",9.7,"[Neil Druckmann, Bruce Straley]","[Troy Baker, Ashley Johnson, Hana Hayes, Je...",68852
3,God of War: Ragnarök,2022,M,"[Action, Adventure, Drama]",9.7,[Eric Williams],"[Christopher Judge, Sunny Suljic, Danielle B...",11538
4,Red Dead Redemption II,2018,M,"[Action, Adventure, Crime]",9.7,None,"[Benjamin Byron Davis, Roger Clark, Rob Wiet...",42785
5,The Witcher 3: Wild Hunt,2015,M,"[Action, Adventure, Drama]",9.7,[Konrad Tomaszkiewicz],"[Doug Cockle, Denise Gough, Jo Wyatt, Jaimi...",29095
6,Mass Effect: Legendary Edition,2021,M,"[Action, Adventure, Drama]",9.6,"[Casey Hudson, Mac Walters]","[Steven Barr, Kimberly Brooks, Keith David, ...",2209
7,Metal Gear Solid,1998,M,"[Action, Adventure, Drama]",9.6,[Hideo Kojima],"[David Hayter, Patric Zimmerman, Peter Lurie...",12111
8,The Legend of Zelda: Ocarina of Time,1998,E,"[Action, Adventure, Fantasy]",9.6,"[Eiji Aonuma, Yoichi Yamada, Yoshiaki Koizum...","[Nobuyuki Hiyama, Fujiko Takimoto, Jun Mizus...",9977
9,Grand Theft Auto V,2013,M,"[Action, Comedy, Crime]",9.5,None,"[Shawn Fonteno, Ned Luke, Steven Ogg, Geral...",64452


KeyboardInterrupt: 

In [144]:
#Save CSV
df.to_csv('ToCSV.csv', index=False)

In [ ]:
#with broken code
"""
*****************************************************PLEASE READ*****************************************************
This will open an extra chrome tab just used to parse some of the data that I couldnt get normally from the website 
directly because it parsed back "None" Values for some elements even though it existed, therefor I couldn't get all the data 
I wanted to collect for this project. the chrome tab allows me read those without restrictions and it solves 
the "None" Value problem

IMPORTANT: Please dont close the tab or the project will break, the tab will close automatically when it is done loading.
"""
driver = webdriver.Chrome()
url = 'https://www.imdb.com/search/title/?title_type=video_game&sort=user_rating,desc'
# url = 'https://www.imdb.com/search/title/?title_type=video_game&sort=year,asc'
# driver.maximize_window()
driver.get(url)
content = driver.page_source.encode('utf-8').strip()
""""""
soup = BeautifulSoup(content,"html.parser")

def data_create():
    data = {
        'title': [],
        'release_date': [],
        'maturity_rating': [],
        'genre': [],
        'rating': [],
        'directors': [],
        'stars': [],
        'votes': []
    }
    global df
    df = pd.DataFrame(data= data)
data_create()


#func used to extract data by inputting the soup and the thing you want to find => "selector"
def extract_data(soup, selector, class_selector):
    try:
        return soup.find(selector, class_= class_selector).text.strip()
    except AttributeError:
        return None


while True:
    game_info = soup.find_all("div", class_="lister-item-content")
    
    for i in game_info:
        #gets game name
        try:
            game_name = (i.find("h3", class_="lister-item-header")).a.text.strip()
        except AttributeError:
            game_name = None
        
        #gets release date
        game_release_date = extract_data(i, "span", "lister-item-year text-muted unbold")
        game_release_date = game_release_date.replace("(", "")
        game_release_date = game_release_date.replace(")", "")
        game_release_date = int(game_release_date.replace("Video Game", "").strip())
        
        #gets maturity rating
        game_maturity = extract_data(i, "span", "certificate")

        #gets genre and split it into an array
        try:
            game_genre = extract_data(i, "span", "genre")
            game_genre = np.array(list(game_genre.split(",")))
        except:
            game_genre = None
            
        #get rating
        try:
            game_rating = float(i.find("div", class_="inline-block ratings-imdb-rating").strong.text.strip())
        except AttributeError:
            game_rating = None
        
        #get Director & Stars and will split them
        #make variables incase they arent found
        
        #get both
        game_director_and_stars = extract_data(i, "p", "")
        game_director, separator, game_stars = game_director_and_stars.partition("|")
        
        #Director found
        try:
            director_or_star_text, separator, game_director = game_director.partition(":")
            #Something Found, Going to check if its Director
            if director_or_star_text.find("Director")!=-1:
                game_director = game_director.strip()
                game_director = ''.join(game_director.splitlines())
                #chekcs if there are multiple directors
                if game_director.find(",")!=1:
                    game_director = list(game_director.split(","))

                #Stars & Director found
                try:
                    stars_text, separator, game_stars = game_stars.partition(":")
                    if stars_text.find("Star")!=-1:
                        game_stars = game_stars.strip()
                        game_stars = ''.join(game_stars.splitlines())
                        game_stars = list(game_stars.split(","))
                #Director found, Star not found
                except:
                    game_stars = None
            #Director not found, going to check for Star
            elif director_or_star_text.find("Star")!=-1:
                game_stars = game_stars.strip()
                game_stars = ''.join(game_stars.splitlines())
                #chekcs if there are multiple stars
                if game_stars.find(",")!=1: 
                    game_stars = list(game_stars.split(","))
            #neither found
            else:
                game_director = None
                game_stars = None

        #neither found
        except:
            game_director = None
            game_stars = None
        
#         #split Director and put it in array
#         deleted_text, separator, game_director = game_director.partition(":")
#         game_director = game_director.strip()
#         game_director = ''.join(game_director.splitlines())
#         game_director = np.array(list(game_director.split(",")))
        
#         #Stars
#         deleted_text, separator, game_stars = game_stars.partition(":")
#         game_stars = game_stars.strip()
#         game_stars = ''.join(game_stars.splitlines())
#         game_stars = np.array(list(game_stars.split(",")))
        
        
        #get votes
        try:
            game_votes = extract_data(i, "p", "sort-num_votes-visible")
            game_votes = ''.join(game_votes.splitlines())
            deleted_text, separator, game_votes = game_votes.partition(":")
            game_votes = int(game_votes.replace(",", "").strip())
        except:
            game_votes = None
        
        
        #adds game details to dataframe
        df.loc[len(df.index)] = [game_name, game_release_date, game_maturity, game_genre, game_rating, game_director, game_stars, game_votes]
        display(df)
    
    
    #find button to click on next page
    uncompleted_next_page_url = soup.find("a", class_="lister-page-next next-page").get("href")
    completed_next_page_url = "https://www.imdb.com/" + uncompleted_next_page_url
    
    print(completed_next_page_url)

    #replaces old url to completed_next_page_url
    url = completed_next_page_url
    driver.get(url)
    content = driver.page_source.encode('utf-8').strip()
    
#     html_request = requests.get(url).text
    soup = BeautifulSoup(content, "html.parser")


driver.quit()

In [188]:
#broken
#Director found
try:
    director_or_star_text, separator, game_director = game_director.partition(":")
    #Something Found, Going to check if its Director
    if director_or_star_text.find("Directors")!=-1:
        game_director = game_director.strip()
        game_director = ''.join(game_director.splitlines())
        #chekcs if there are multiple directors
        if game_director.find(",")!=1:
            game_director = list(game_director.split(","))
        
        #Stars & Director found
        try:
            stars_text, separator, game_stars = game_stars.partition(":")
            if stars_text.find("Stars")!=-1:
                game_stars = game_stars.strip()
                game_stars = ''.join(game_stars.splitlines())
                game_stars = list(game_stars.split(","))
        #Director found, Star not found
        except:
            game_stars = None
    #Director not found, going to check for Star
    elif director_or_star_text.find("Stars")!=-1:
        game_stars = game_stars.strip()
        game_stars = ''.join(game_stars.splitlines())
        #chekcs if there are multiple stars
        if game_stars.find(",")!=1: 
            game_stars = list(game_stars.split(","))
    #neither found
    else:
        game_director = None
        game_stars = None

#neither found
except:
    game_director = None
    game_stars = None

    
print(f"director: {game_director}")
print(f"stars: {game_stars}")

director: ['Matthew Gallant', ' Bruce Straley']
stars: ['Ashley Johnson', ' Troy Baker', ' Hana Hayes', ' Jeffrey Pierce']
